In [9]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import DataLoader

import torchvision

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# MNIST Dataset
train_data = torchvision.datasets.MNIST(
    root='data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

test_data = torchvision.datasets.MNIST(
    root='data',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 11538600.22it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 28889981.83it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|███████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 8747527.41it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [5]:
print(f'Shape of train data: {train_data.data.shape}')
print(f'Shape of test data: {test_data.data.shape}')

Shape of train data: torch.Size([60000, 28, 28])
Shape of test data: torch.Size([10000, 28, 28])


In [8]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [12]:
train_dataloader = DataLoader(
    train_data,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_dataloader = DataLoader(
    test_data,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [23]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [24]:
model = Net().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [35]:
n_epochs = 10

In [38]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print(f'Train epoch: {epoch}, [{batch_idx*len(data)}/{len(train_dataloader.dataset)}, ({100*batch_idx/len(train_dataloader):.3f}%)], loss: {loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_dataloader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_dataloader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_dataloader.dataset)} ({correct/(len(test_dataloader.dataset)):.3f}%)')

In [39]:
for epoch in range(1, n_epochs+1):
    train(epoch)
    test()

C:\Users\Dong\AppData\Local\Temp\ipykernel_6540\1616078874.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train epoch: 1, [0/60000, (0.000%)], loss: 0.295317
Train epoch: 1, [1280/60000, (2.132%)], loss: 0.284774
Train epoch: 1, [2560/60000, (4.264%)], loss: 0.342401
Train epoch: 1, [3840/60000, (6.397%)], loss: 0.350270
Train epoch: 1, [5120/60000, (8.529%)], loss: 0.580023
Train epoch: 1, [6400/60000, (10.661%)], loss: 0.398987
Train epoch: 1, [7680/60000, (12.793%)], loss: 0.207298
Train epoch: 1, [8960/60000, (14.925%)], loss: 0.397964
Train epoch: 1, [10240/60000, (17.058%)], loss: 0.286570
Train epoch: 1, [11520/60000, (19.190%)], loss: 0.175699
Train epoch: 1, [12800/60000, (21.322%)], loss: 0.340903
Train epoch: 1, [14080/60000, (23.454%)], loss: 0.178692
Train epoch: 1, [15360/60000, (25.586%)], loss: 0.298882
Train epoch: 1, [16640/60000, (27.719%)], loss: 0.290195
Train epoch: 1, [17920/60000, (29.851%)], loss: 0.274759
Train epoch: 1, [19200/60000, (31.983%)], loss: 0.307639
Train epoch: 1, [20480/60000, (34.115%)], loss: 0.220385
Train epoch: 1, [21760/60000, (36.247%)], loss:

In [42]:
torch.save(model.state_dict(), './model.pth')
torch.save(optimizer.state_dict(), './optimizer.pth')